# SETUP

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [ ]:
!pip install ultralytics


import ultralytics
from ultralytics import YOLO

In [ ]:
!pip install supervision
import supervision as sv


In [ ]:
import numpy as np

In [ ]:
import pandas as pd

In [ ]:
import cv2

In [ ]:
!pip install trackers
import trackers

## GOOGLE DRIVE MOUNT

In [ ]:
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive')

Mounted at /content/drive


# RESTORE DATA

In [ ]:
%cd "/content"
!cp "/content/drive/MyDrive/Colab Notebooks/MVI_1482_VIS.avi" .

!cp "/content/drive/MyDrive/Colab Notebooks/maritime-best.pt" .

!cp "/content/drive/MyDrive/Colab Notebooks/gt.txt" .


/content


In [ ]:
SOURCE_VIDEO_PATH = "/content/drive/MyDrive/Colab Notebooks/MVI_1482_VIS.avi"

In [ ]:
MODEL_PATH = "/content/maritime-best.pt"

# Model Import

In [ ]:
model = YOLO(MODEL_PATH)


In [ ]:
# dict maping class_id to class_name
CLASS_NAMES_DICT = model.model.names
print(CLASS_NAMES_DICT)
# class ids matching the class names we have chosen // Ferry only
SELECTED_CLASS_IDS = [2]

{0: 'Boat', 1: 'Buoy', 2: 'Ferry', 3: 'Flying bird-plane', 4: 'Kayak', 5: 'Other', 6: 'Sail boat', 7: 'Speed boat', 8: 'Vessel-ship'}


In [ ]:
# dict maping class_id to class_name
CLASS_NAMES_DICT = model.model.names




# MODEL FRAME TEST

In [ ]:
# create frame generator
generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)
# create instance of BoxAnnotator and LabelAnnotator
box_annotator = sv.BoxAnnotator(thickness=4)
label_annotator = sv.LabelAnnotator(text_thickness=2, text_scale=1.5, text_color=sv.Color.BLACK)
# acquire first video frame
iterator = iter(generator)
frame = next(iterator)
# model prediction on single frame and conversion to supervision Detections
results = model(frame, verbose=False)[0]

# convert to Detections
detections = sv.Detections.from_ultralytics(results)
# only consider class id from selected_classes define above
detections = detections[np.isin(detections.class_id, SELECTED_CLASS_IDS)]

# format custom labels
labels = [
    f"{CLASS_NAMES_DICT[class_id]} {confidence:0.2f}"
    for confidence, class_id in zip(detections.confidence, detections.class_id)
]

# annotate and display frame
annotated_frame = frame.copy()
annotated_frame = box_annotator.annotate(
    scene=annotated_frame, detections=detections)
annotated_frame = label_annotator.annotate(
    scene=annotated_frame, detections=detections, labels=labels)

%matplotlib inline
sv.plot_image(annotated_frame, (16, 16))

# TRACK

In [ ]:
!pip install -q deep_sort_realtime
from deep_sort_realtime.deepsort_tracker import DeepSort

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 73.4 MB/s eta 0:00:00


In [ ]:
from deep_sort_realtime.deepsort_tracker import DeepSort

def process_video_with_deepsort(video_path,
                                model,
                                output_file,
                                video_output_path):
    """
    YOLOv8 + deep_sort_realtime
    - ids 1,2,3...
    - MOT-Challenge txt
    - Annotated mp4
    """

    # ---------- 1) Tracker ---------- #
    deepsort = DeepSort(
        max_age=70,
        n_init=3,
        nms_max_overlap=1.0,
        max_cosine_distance=0.3,
        embedder="mobilenet",
        half=True,
        bgr=True,
        embedder_gpu=True
    )

    # ---------- 2) Video I/O -------- #
    cap  = cv2.VideoCapture(video_path)
    w, h = int(cap.get(3)), int(cap.get(4))
    fps  = int(cap.get(5))

    out = cv2.VideoWriter(video_output_path,
                          cv2.VideoWriter_fourcc(*'mp4v'),
                          fps, (w, h))

    box_annot   = sv.BoxAnnotator()
    label_annot = sv.LabelAnnotator(text_thickness=2, text_scale=1.5,
                                    text_color=sv.Color.BLACK)

    frame_id, next_id = 1, 1
    id_remap = {}                # {raw_id : 1…N}

    # ---------- 3) Loop ------------ #
    with open(output_file, "w") as f:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # ---- 3.1  YOLO ---------- #
            res  = model(frame, verbose=False)[0]
            dets = sv.Detections.from_ultralytics(res)

            mask = np.isin(dets.class_id, SELECTED_CLASS_IDS)
            dets = dets[mask]
            dets = dets[dets.confidence > 0.5]

            # ---- 3.2  DeepSort Entry #
            ds_inp = []
            for xyxy, conf, cls in zip(dets.xyxy,
                                       dets.confidence,
                                       dets.class_id):
                x1,y1,x2,y2 = xyxy
                ds_inp.append(([float(x1),
                                float(y1),
                                float(x2 - x1),
                                float(y2 - y1)],
                               float(conf),
                               int(cls)))

            # ---- 3.3 Tracking ---------- #
            tracks = deepsort.update_tracks(ds_inp, frame=frame)

            draw_boxes, draw_ids, draw_confs = [], [], []
            for trk in tracks:
                if not trk.is_confirmed():
                    continue
                x1,y1,x2,y2 = trk.to_ltrb()        # XYXY
                raw_id      = trk.track_id
                conf        = trk.det_conf if trk.det_conf else 1.0

                # id remap → 1,2,3…
                if raw_id not in id_remap:
                    id_remap[raw_id] = next_id
                    next_id += 1
                new_id = id_remap[raw_id]

                # MOT satırı
                f.write(f"{frame_id},{new_id},{x1:.2f},{y1:.2f},"
                        f"{(x2 - x1):.2f},{(y2 - y1):.2f},"
                        f"{conf:.4f},-1,-1,-1\n")

                draw_boxes.append([x1,y1,x2,y2])
                draw_ids.append(new_id)
                draw_confs.append(conf)

            # ---- 3.4  Drawing ---------- #
            if draw_boxes:
                det_draw = sv.Detections(
                    xyxy=np.array(draw_boxes, dtype=np.float32),
                    confidence=np.array(draw_confs, dtype=np.float32),
                    class_id=np.zeros(len(draw_boxes), dtype=int),
                    tracker_id=np.array(draw_ids, dtype=int)
                )
                labels = [f"#{i} Ferry ({c:.2f})"
                          for i, c in zip(draw_ids, draw_confs)]
                frame  = box_annot.annotate(frame, det_draw)
                frame  = label_annot.annotate(frame, det_draw, labels)

            out.write(frame)
            frame_id += 1

    cap.release()
    out.release()
    return id_remap

## Run


In [ ]:
!pip install --only-binary=numpy numpy==1.26.4

In [ ]:
%cd /content/
# Run DeepSORT tracking
print("\nRunning DeepSORT tracking...")
deepsort_id_map = process_video_with_deepsort(
    SOURCE_VIDEO_PATH,
    model,
    'deepsort_results.txt',
    'deepsort_tracking_output.mp4'
)



## Save video outputs to GDrive and Print

In [ ]:
output_dir = "/content/drive/MyDrive/Colab Notebooks/deepsort"
os.makedirs(output_dir, exist_ok=True)

# Save DeepSORT outputs to GDrive
!cp deepsort_tracking_output.mp4 "/content/drive/MyDrive/Colab Notebooks/deepsort/"
!cp deepsort_results.txt "/content/drive/MyDrive/Colab Notebooks/deepsort/"

# Check DeepSORT results
df_sort = pd.read_csv('deepsort_results.txt', header=None,
                     names=['frame', 'id', 'bb_left', 'bb_top', 'bb_width', 'bb_height', 'conf', 'x', 'y', 'z'])

print("\nDeepSORT Stats:")
print(f"Total frame count: {df_sort['frame'].nunique()}")
print(f"Total track count: {df_sort['id'].nunique()}")
print("\nTrack ID counts:")
print(df_sort.groupby('id').size())